In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.preprocessing import PowerTransformer, RobustScaler
from sklearn.mixture import BayesianGaussianMixture
import sys

start = 300
end = 400

title = f'BGM_{start}_{end}_pt_rs.csv'
print(title)



BGM_300_400_pt_rs.csv


In [15]:
data = pd.read_csv('../data/data.csv')

best_data =[
'f_07','f_08', 'f_09', 'f_10',
'f_11', 'f_12', 'f_13', 'f_22',
'f_23', 'f_24', 'f_25','f_26',
'f_27', 'f_28']


rob_scaler = RobustScaler().fit(data)
power_transformer = PowerTransformer().fit(data)

data_scaled = rob_scaler.fit_transform(data[best_data])
data_scaled = power_transformer.fit_transform(data_scaled)
data_scaled = pd.DataFrame(data_scaled, columns = best_data)

values = [0,1,2,3,4,5,6]
pred_test = pd.DataFrame(np.zeros((data_scaled.shape[0],7)), columns = values)

for seed in tqdm(range(start,end)):
    
    df = pd.DataFrame(index = data.index)
    gmm = BayesianGaussianMixture(
            n_components=7,
            random_state = seed,
            tol = 0.01,
            covariance_type = 'full',
            max_iter = 100,
            n_init=3
          )
    
    # fitting and probability prediction
    gmm.fit(data_scaled)
    pred_seed = gmm.predict_proba(data_scaled) # predict_proba for probabilities
    
    # the clusters prediction for the current seed :
    MAX = np.argmax(pred_seed, axis=1)
    df[f'pred_{seed}'] = MAX
    
    # Sort of the prediction by same value of cluster (for addition of every seed)
    pred_keys = df[f'pred_{seed}'].value_counts().index.tolist()
    pred_dict = dict(zip(pred_keys, values))
    df[f'pred_{seed}'] = df[f'pred_{seed}'].map(pred_dict)

    pred_new = pd.DataFrame(pred_seed).rename(columns = pred_dict)
    pred_new = pred_new.reindex(sorted(pred_new.columns), axis=1)
    pred_test += pred_new # Soft voting by probabiliy addition


 82%|████████▏ | 82/100 [1:55:26<24:13, 80.76s/it]  /opt/conda/lib/python3.7/site-packages/sklearn/mixture/_base.py:282: ConvergenceWarning: Initialization 3 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  ConvergenceWarning,
100%|██████████| 100/100 [2:21:03<00:00, 84.64s/it]


In [1]:

predictions = np.argmax(np.array(pred_test), axis=1)

submission = pd.read_csv("../data/sample_submission.csv")
submission["Predicted"] = predictions
submission.to_csv(title,index = False)

NameError: name 'np' is not defined

In [ ]:
pred_test

,0,1,2,3,4,5,6
0,0.180931,0.765457,23.105141,171.049862,0.084482,0.835186,3.978941
1,3.338697,20.191851,4.619448,0.109863,4.747797,25.178192,141.814153
2,0.479187,32.369766,15.829796,150.666120,0.000581,0.026640,0.627909
3,6.532555,17.008652,1.989894,3.434521,167.494449,1.258156,2.281773
4,195.545403,0.005106,0.971224,0.001775,0.030284,0.368148,3.078060
...,...,...,...,...,...,...,...
97995,26.976953,42.575515,39.403595,7.797493,1.410023,8.653080,73.183341
97996,82.944408,15.072784,94.268078,0.124356,0.088718,2.230097,5.271560
97997,13.811603,164.480569,1.394257,17.771208,0.806170,0.307563,1.428630
97998,0.470525,26.484183,7.124176,3.686405,8.236522,125.020016,28.978172
